# DAMP: Discord-Aware Matrix Profile

Authors in [DAMP](https://www.cs.ucr.edu/~eamonn/DAMP_long_version.pdf) presented a method for discord detection that is scalable and it can be used in offline and online mode.

To better understand the mechanism behind this method, we should first understand the difference between the full matrix profile and the left matrix profile of a time series `T`. For a subsequence with length `m`, and the start index `i`, i.e. `S_i = T[i:i+m]`, there are two groups of neighbors, known as left and right neighbors. The left neighbors are the subsequences on the left side of `S_i`, i.e. the subsequences in `T[:i]`. And, the right neighbors are the subsequences on the right side of `S_i`, i.e. the subsequences in `T[i+1:]`. The `i`-th element of the full matrix profile is the minimum distance between `S_i` and all of its neighbors, considering both left and right ones. However, in the left matrix profile, the `i`-th element is the minimum distance between the subsequence `S_i` and its left neighbors. In both cases, it is recommended to avoid considering subsequences that are close to `S_i`, known as trivial neighbors.

One can use either the full matrix profile or the left matrix profile to find the top discord, a subsequence whose distance to its nearest neighbor is larger than the distance of any other subsequences to their nearest neighbors. However, using full matrix profile for detecting discords might result in missing the case where there are two rare subsequences that happen to also be similar to each other (a case that is known as "twin freak"). On the other hand, the left matrix profile resolves this problem by capturing the discord at its first occurance. Hence, even if there are two or more of such discords, we can still capture the first occurance by using the left matrix profile.

The original `DAMP` algorithm needs a parameter called `split_idx`. For a given `split_idx`, the train part is `T[:split_idx]` and the potential anomalies should be coming from `T[split_idx:]`. The value of `split_idx` is problem dependent. If `split_idx` is too small, then `T[:split_idx]` may not contain all different kinds of regular patterns. Hence, we may incorrectly select a subsequence as a discord. If `split_idx` is too large, we may miss a discord if that discord and its nearest neighbor are both in `T[:split_idx]`. The following two extreme scenarios can help with understanding the importance of choosing proper `split_idx`.

(1) `split_idx = 0`: In this case, the first subsequence can be a discord itself as it is a "new" pattern. <br>
(2) `split_idx = len(T) - m` In such case, the last pattern is the only pattern that will be analyzed for the discord. It will be compared against all previous subsequences!

As we can see, the problem-dependent parameter `split_idx` should be set to a proper value by the user.

# Getting Started

In [2]:
import math
import time

import numpy as np
import matplotlib.pyplot as plt
import stumpy

from stumpy import core
from scipy.io import loadmat

## Naive approach

In [3]:
def naive_DAMP(T, m, split_idx):
    """
    Compute the top-1 discord in `T`, where the subsequence discord resides in T[split_index:]
    
    Parameters
    ----------
    T : numpy.ndarray
        A time series for which the top discord will be computed.
        
    m : int
        Window size
    
    split_idx : int
        The split index between train and test. See note below for further details.
    
    Returns
    -------
    PL : numpy.ndarry
        The (exact) left matrix profile. All infinite distances are ingored when computing
        the discord.
        
    discord_dist : float
        The discord's distance, which is the distance between the top discord and its
        left nearest neighbor
        
    discord_idx : int
        The start index of the top discord
    
    """
    mp = stumpy.stump(T, m)
    IL = mp[:, 2].astype(np.int64)
    
    k = len(T) - m + 1  # len(IL)
    PL = np.full(k, np.inf, dtype=np.float64)
    for i in range(split_idx, k):
        nn_i = IL[i]
        if nn_i >= 0:
            PL[i] = np.linalg.norm(core.z_norm(T[i : i + m]) - core.z_norm(T[nn_i : nn_i + m]))
    
    # finding the max of PL, while ignoring infinite values
    discord_idx = np.argmax(
        np.where(PL==np.inf, np.NINF, PL)
    )
    discord_dist = PL[discord_idx]
    if discord_dist == np.NINF:
        discord_dist = np.inf
        discord_idx = -1
        
    return PL, discord_dist, discord_idx

## DAMP

In [4]:
def next_pow2(v):
    """
    Compute the smallest "power of two" number that is greater than/ equal to `v`
    
    Parameters
    ----------
    v : float
        A real positive value
    
    Returns
    -------
    out : int
        An integer value that is power of two, and satisfies `out >= v`
    """
    return int(math.pow(2, math.ceil(math.log2(v))))

In [5]:
def naive_get_range_damp(stop, m, start=0):
    """
    For the given range `(start, stop)`, segmentize it into 
    chunks, each with a length of power of two. Each chunk is
    preceded by one that is twice in size. The last chunk (i.e.
    the left-most chunk) starts at `start`.
    
    Parameters
    ----------
    stop : int
        The stop index
    
    m : int
        Window size
    
    start : int, default 0
        The start index
    
    Returns
    -------
    out : numpy.ndarray
        A 2D numpy array, where the i-th row shows the 
        range (start, stop) of the i-th chunk. out[0][1]
        must be `stop` and out[-1][0] must be `start`. 
    """
    chunksize = next_pow2(m)
    
    out = []
    chunk_stop = stop
    while start < chunk_stop:
        chunk_start = max(chunk_stop - chunksize, start)
        out.append([chunk_start, chunk_stop])
        
        if chunk_start == start:
            break
        
        # The chunk_stop of the preceding chunk
        # is `chunk_start` but shifted by `m` to 
        # include new subsequences that end after
        # current `chunk_start`.
        chunk_stop = chunk_start + m
        chunksize = 2 * chunksize
    
    return np.array(out)

In [6]:
def _backward_process(
    T, 
    m, 
    query_idx, 
    excl_zone,
    M_T, 
    Σ_T, 
    T_subseq_isconstant, 
    bsf,
    chunks_range=None,
):
    """
    Compute the (approximate) distance between the subsequence `T[query_idx:query_idx+m]`
    and its nearest neighbor, and update the best-so-far discord distance. 
    
    Parameters
    ----------
    T : numpy.ndarray
        A time series
    
    m : int
        Window size
    
    query_idx : int
        The start index of the query with length `m`, i.e. `T[query_idx:query_idx+m]`
    
    excl_zone : int
        Size of the exclusion zone
        
    M_T : np.ndarray
        The sliding mean of `T`
        
    Σ_T : np.ndarray
        The sliding standard deviation of `T`
    
    T_subseq_isconstant : numpy.ndarray
        A numpy boolean array whose i-th element indicates whether the subsequence
        `T[i : i+m]` is constant (True)
    
    bsf : float
        The best-so-far discord distance
        
    chunks_range : numpy.ndarray, default None
        A 2D numpy array consisting of rows, each represents
        the (start, stop) range of a chunk
        
    Returns
    -------
    nn_distance : float
        The (approximate) left matrix profile value that corresponds to 
        the query, `T[query_idx : query_idx + m]`.
    
    bsf : float
        The best-so-far discord distance 
    """
    if chunks_range is None:
        # The avoid the trivial (left) neighbors of `T[query_idx : query_idx+m]`,
        # the stop index of non-trivial subsequences is `query_idx - excl_zone + m`
        chunks_range = naive_get_range_damp(query_idx - excl_zone + m, m, start=0)
    
    nn_distance = np.inf
    for (start, stop) in chunks_range:
        QT = core.sliding_dot_product(
            T[query_idx : query_idx + m], 
            T[start : stop],
        )
        D = core._mass(
            T[query_idx : query_idx + m],
            T[start : stop],
            QT=QT,
            μ_Q=M_T[query_idx],
            σ_Q=Σ_T[query_idx],
            M_T=M_T[start : stop - m + 1],
            Σ_T=Σ_T[start : stop - m + 1],
            Q_subseq_isconstant=T_subseq_isconstant[query_idx],
            T_subseq_isconstant=T_subseq_isconstant[start : stop - m + 1],
            )
        
        nn_distance = min(nn_distance, np.min(D))
        if nn_distance < bsf:
            break
    
    bsf = max(bsf, nn_distance)
    
    return nn_distance, bsf

In [7]:
def _foreward_process(
    T, 
    m,  
    excl_zone,
    M_T, 
    Σ_T, 
    T_subseq_isconstant, 
    query_idx,
    lookahead,
    PL,
):
    """
    Update the (approximate) left matrix profile `PL` in place by checking the right neighbors of
    `T[query_idx : query_idx + m]`
    
    Paramaters
    ----------
    T : numpy.ndarray
        A time series
    
    m : int
        Window size
    
    excl_zone : int
        Size of the exclusion zone
    
    M_T : numpy.ndarray
        The sliding mean
        
    Σ_T : numpy.ndarray
        The sliding standard deviation
        
    T_subseq_isconstant : numpy.ndarray
        A numpy boolean array whose i-th element indicates whether the subsequence
        `T[i : i+m]` is constant (True)
 
    query_idx : int
        The start index of the query with length `m`, i.e. `T[query_idx:query_idx+m]`
    
    lookahead : int
        The size of chunk whose subsequences are compared with the query.
    
    PL : numpy.ndarray
        A 1D numpy array that contains the (approximate) values of
        the left matrix profile.
    
    Returns
    -------
    None
    """    
    start = query_idx + excl_zone + 1
    stop = min(start + lookahead, len(T))
    
    if stop - start >= m:
        QT = core.sliding_dot_product(T[query_idx : query_idx + m], T[start : stop])
        D = core._mass(
            T[query_idx : query_idx + m],
            T[start : stop],
            QT=QT,
            μ_Q=M_T[query_idx],
            σ_Q=Σ_T[query_idx],
            M_T=M_T[start : stop - m + 1],
            Σ_T=Σ_T[start : stop - m + 1],
            Q_subseq_isconstant=T_subseq_isconstant[query_idx],
            T_subseq_isconstant=T_subseq_isconstant[start : stop - m + 1],
        )
        
        PL[start : stop - m + 1] = np.minimum(PL[start : stop - m + 1], D)

    return

In [8]:
def DAMP(T, m, split_idx): 
    """
    Compute the approximate left matrix profile for `T`. The global maximum
    of this approximate profile is exact.
    
    Parameters
    ----------
    T : numpy.ndarray
        A time series of interest
    
    m : int
        Window size
    
    split_idx : int
        The location of split point between train and test. The data `T[:split_idx]`
        is considered as the train set and the remaining, i.e. `T[split_idx:]` is test.
    
    Returns
    -------
    PL : numpy.ndarry
        The (approximate) left matrix profile. The finite global maximum of `PL` is exact,
        and its corresponding index indicates the location of discord.
        
    discord_dist : float
        The distance between the discord and its nearest neighbor.
        
    discord_idx : int
        The start index of discord.
    """
    T, M_T, Σ_T, T_subseq_isconstant = core.preprocess(T, m)
     
    l = len(T) - m + 1
    PL = np.full(l, np.inf, dtype=np.float64) 
    
    excl_zone = int(math.ceil(m / stumpy.core.config.STUMPY_EXCL_ZONE_DENOM))
    chunks_range = naive_get_range_damp(split_idx - excl_zone + m, m, start=0)
    
    # Compute the last chunk size, and the expected size of the last chunk. The latter
    # is basically the `n_chunks`-th item in a geometric series with coeficient of
    # `next_pow2(m)` and the ratio of `2`.
    last_chunk_size = chunks_range[-1][1] - chunks_range[-1][0]
    expected_last_chunk_size =  next_pow2(m) * (2 ** (len(chunks_range) - 1)) 
    
    bsf = np.NINF  
    lookahead = next_pow2(m)
    for i in range(split_idx, l):
        if PL[i] < bsf:
            continue 
        
        PL[i], bsf = _backward_process(
            T, 
            m, 
            i, 
            excl_zone, 
            M_T, 
            Σ_T, 
            T_subseq_isconstant, 
            bsf, 
            chunks_range=chunks_range
        )
        _foreward_process(T, m, excl_zone, M_T, Σ_T, T_subseq_isconstant, i, lookahead, PL)
        
        # update chunks_range
        chunks_range = chunks_range + 1 
        if last_chunk_size < expected_last_chunk_size:
            chunks_range[-1][0] = 0 
            last_chunk_size += 1
        else:
            new_chunk = np.array([
                [0, chunks_range[-1][0] + m]
            ])
            chunks_range = np.append(chunks_range, new_chunk, axis=0)
            expected_last_chunk_size = 2 * expected_last_chunk_size
            last_chunk_size = 1 
            
    P = np.where(PL==np.inf, np.NINF, PL)
    discord_idx = np.argmax(P)
    discord_dist = P[discord_idx]
    if discord_dist == np.NINF:
        discord_dist = np.inf
        discord_idx = -1
        
    return PL, discord_dist, discord_idx

## Example

In [9]:
seed = 100
np.random.seed(seed)

T = np.random.rand(100000)
m = 50
split_index = 200

In [10]:
naive_DAMP(T, m, split_index)  # dummy run
t_start = time.time()
excl_zone_denom = core.config.STUMPY_EXCL_ZONE_DENOM
core.config.STUMPY_EXCL_ZONE_DENOM = 1.0
PL, discord_dist, discord_index = naive_DAMP(T, m, split_index)
core.config.STUMPY_EXCL_ZONE_DENOM = excl_zone_denom
t_end = time.time()

print('discord_dist: ', discord_dist)
print('discord_index: ', discord_index)
print('running time [sec]: ', t_end - t_start)

discord_dist:  8.500883427933502
discord_index:  209
running time [sec]:  8.726994037628174


In [11]:
DAMP(T, m, split_index)  # dummpy run

t_start = time.time()
excl_zone_denom = core.config.STUMPY_EXCL_ZONE_DENOM
core.config.STUMPY_EXCL_ZONE_DENOM = 1
PL, discord_score, discord_idx = DAMP(T, m, split_index)
core.config.STUMPY_EXCL_ZONE_DENOM = excl_zone_denom
t_end = time.time()

print('discord_dist: ', discord_score)
print('discord_index: ', discord_idx)
print('running time [sec]: ', t_end - t_start)

discord_dist:  8.500883427933504
discord_index:  209
running time [sec]:  2.205477237701416
